<a href="https://colab.research.google.com/github/RaphaelCarvalh/BootCampAVANTI_machine_learning/blob/ativ04-et01-analise-dataset/Et01_analise_dataset_fase2_pr%C3%A9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Projeto Clothing Co-Parsing - Etapa 1 - Notebook: Análise do Dataset - TIV-04-ET-02


In [ ]:
# ============================================================
# Projeto Clothing Co-Parsing - Notebook Principal (Professor)
# ============================================================

# 1. Instalação de pacotes necessários
print("Instalando pacotes necessários...")
!pip install opendatasets pandas matplotlib opencv-python pillow --quiet
print("Pacotes instalados com sucesso.\n")

# 2. Importações iniciais
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import cv2
from PIL import Image
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import time, copy, json
from tqdm import tqdm

print("Pacotes importados com sucesso.\n")

# Otimização no CUDA
torch.backends.cudnn.benchmark = True


Instalando pacotes necessários...
Pacotes instalados com sucesso.



In [ ]:
# 3. Montar Google Drive e carregar dataset já limpo
from google.colab import drive
drive.mount('/content/drive')

LOAD_PATH = "/content/drive/MyDrive/ClothingDataset/df_clean.csv"
df = pd.read_csv(LOAD_PATH)

print("Exemplo do dataset carregado:")
print(df.head())
print(f"Total de imagens: {len(df)}\n")

# Número de classes
num_classes = len(df['label'].unique())
print(f"Número de classes: {num_classes}")


In [20]:
# 4. Transformações e Augmentations
print("Configurando transformações e augmentations...\n")

IMG_SIZE = 224

train_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.RandomResizedCrop(IMG_SIZE),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

val_test_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

print("Transformações configuradas com sucesso.")


Configurando transformações e augmentations...

Transformações configuradas com sucesso.


In [21]:
# 5. Dataset personalizado
class ClothingDataset(Dataset):
    def __init__(self, data, transform):
        self.data = data
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path, label = self.data[idx, 0], self.data[idx, 1]
        img = cv2.imread(img_path)
        if img is None:
            # fallback para imagens corrompidas
            img = np.zeros((IMG_SIZE, IMG_SIZE, 3), dtype=np.uint8)
        else:
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        if self.transform:
            img = self.transform(img)
        return img, label

# 6. Split em treino, validação, teste
train_data, temp_data = train_test_split(df.values, test_size=0.2, random_state=42, stratify=df.values[:,1])
val_data, test_data = train_test_split(temp_data, test_size=0.5, random_state=42, stratify=temp_data[:,1])

# Criar datasets
train_ds = ClothingDataset(train_data, transform=train_transform)
val_ds = ClothingDataset(val_data, transform=val_test_transform)
test_ds = ClothingDataset(test_data, transform=val_test_transform)

# Criar dataloaders
BATCH_SIZE = 32
train_dl = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
val_dl = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False)
test_dl = DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=False)

print(f"Treino: {len(train_ds)} | Validação: {len(val_ds)} | Teste: {len(test_ds)}")


Treino: 1676 | Validação: 210 | Teste: 210


In [27]:
def train_model(
    model,
    train_dl,
    val_dl,
    criterion,
    optimizer,
    num_epochs=10,
    device=None,
    save_path="best_model.pth",
    early_stopping_patience=None,
    grad_clip=None,
    use_amp=True
):
    if device is None:
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    scaler = torch.cuda.amp.GradScaler() if (use_amp and device.type == "cuda") else None

    history = {"train_loss": [], "train_acc": [], "val_loss": [], "val_acc": []}
    best_model_wts = copy.deepcopy(model.state_dict())
    best_val_acc = 0.0
    patience_counter = 0

    for epoch in range(1, num_epochs+1):
        print(f"\nÉpoca {epoch}/{num_epochs}")
        start_time = time.time()

        # ---------- Treino ----------
        model.train()
        train_loss, train_correct, total = 0.0, 0, 0
        for inputs, labels in tqdm(train_dl, desc="Treinando", leave=False):
            #inputs, labels = inputs.to(device), torch.tensor(labels, dtype=torch.long).to(device)
            inputs, labels = inputs.to(device), labels.detach().clone().to(device, dtype=torch.long)
            optimizer.zero_grad()

            if scaler:
                with torch.cuda.amp.autocast():
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)
                scaler.scale(loss).backward()
                if grad_clip:
                    scaler.unscale_(optimizer)
                    torch.nn.utils.clip_grad_norm_(model.parameters(), grad_clip)
                scaler.step(optimizer)
                scaler.update()
            else:
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                if grad_clip:
                    torch.nn.utils.clip_grad_norm_(model.parameters(), grad_clip)
                optimizer.step()

            preds = torch.argmax(outputs, 1)
            train_loss += loss.item() * inputs.size(0)
            train_correct += torch.sum(preds == labels).item()
            total += inputs.size(0)

        epoch_loss = train_loss / total
        epoch_acc = train_correct / total
        history["train_loss"].append(epoch_loss)
        history["train_acc"].append(epoch_acc)

        # ---------- Validação ----------
        model.eval()
        val_loss, val_correct, val_total = 0.0, 0, 0
        with torch.no_grad():
            for inputs, labels in val_dl:
                #inputs, labels = inputs.to(device), torch.tensor(labels, dtype=torch.long).to(device)
                inputs, labels = inputs.to(device), labels.detach().clone().to(device, dtype=torch.long)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                preds = torch.argmax(outputs, 1)
                val_loss += loss.item() * inputs.size(0)
                val_correct += torch.sum(preds == labels).item()
                val_total += inputs.size(0)

        val_loss /= val_total
        val_acc = val_correct / val_total
        history["val_loss"].append(val_loss)
        history["val_acc"].append(val_acc)

        print(f"Train Loss: {epoch_loss:.4f} | Train Acc: {epoch_acc:.4f}")
        print(f"Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f}")

        # Salvar melhor modelo
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            best_model_wts = copy.deepcopy(model.state_dict())
            torch.save(best_model_wts, save_path)
            patience_counter = 0
            print(f"✅ Melhor modelo salvo em {save_path}")
        else:
            patience_counter += 1
            if early_stopping_patience and patience_counter >= early_stopping_patience:
                print("⏹️ Early stopping ativado.")
                break

        print(f"⏱️ Tempo da época: {(time.time()-start_time):.1f}s")

    model.load_state_dict(best_model_wts)
    return model, history, best_val_acc


transformações e augmentations

In [28]:
# 6. Definição dos modelos
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Usando dispositivo: {device}")

# ResNet50
resnet50 = models.resnet50(weights="IMAGENET1K_V1")
in_features = resnet50.fc.in_features
resnet50.fc = nn.Linear(in_features, num_classes)
resnet50 = resnet50.to(device)

# EfficientNet-B0
efficientnet = models.efficientnet_b0(weights="IMAGENET1K_V1")
in_features = efficientnet.classifier[1].in_features
efficientnet.classifier[1] = nn.Linear(in_features, num_classes)
efficientnet = efficientnet.to(device)

# VGG16
vgg16 = models.vgg16(weights="IMAGENET1K_V1")
in_features = vgg16.classifier[6].in_features
vgg16.classifier[6] = nn.Linear(in_features, num_classes)
vgg16 = vgg16.to(device)

print("Modelos inicializados com sucesso.")


Usando dispositivo: cpu
Modelos inicializados com sucesso.


Criar Datasets e DataLoaders

In [ ]:
# Critério de perda
criterion = nn.CrossEntropyLoss()

# Treinar ResNet50
optimizer_resnet = optim.Adam(resnet50.parameters(), lr=1e-4)
print("\n Treinando ResNet50...")
resnet50, history_resnet, best_acc_resnet = train_model(
    model=resnet50,
    train_dl=train_dl,
    val_dl=val_dl,
    criterion=criterion,
    optimizer=optimizer_resnet,
    num_epochs=10,
    device=device,
    save_path="best_resnet50.pth",
    early_stopping_patience=3
)






 Treinando ResNet50...

Época 1/10


Train Loss: 0.0271 | Train Acc: 1.0000
Val Loss: 0.0049 | Val Acc: 1.0000
✅ Melhor modelo salvo em best_resnet50.pth
⏱️ Tempo da época: 1149.4s

Época 2/10


Train Loss: 0.0006 | Train Acc: 1.0000
Val Loss: 0.0005 | Val Acc: 1.0000
⏱️ Tempo da época: 1137.9s

Época 3/10


Train Loss: 0.0004 | Train Acc: 1.0000
Val Loss: 0.0003 | Val Acc: 1.0000
⏱️ Tempo da época: 1131.6s

Época 4/10


Train Loss: 0.0003 | Train Acc: 1.0000
Val Loss: 0.0003 | Val Acc: 1.0000
⏹️ Early stopping ativado.

 Treinando EfficientNet-B0...

Época 1/10


Train Loss: 0.1069 | Train Acc: 0.9916
Val Loss: 0.0071 | Val Acc: 1.0000
✅ Melhor modelo salvo em best_efficientnet.pth
⏱️ Tempo da época: 516.5s

Época 2/10


Train Loss: 0.0061 | Train Acc: 1.0000
Val Loss: 0.0000 | Val Acc: 1.0000
⏱️ Tempo da época: 515.2s

Época 3/10


Train Loss: 0.0031 | Train Acc: 1.0000
Val Loss: 0.0050 | Val Acc: 1.0000
⏱️ Tempo da época: 512.8s

Época 4/10


Train Loss: 0.0020 | Train Acc: 1.0000
Val Loss: 0.0049 | Val Acc: 1.0000
⏹️ Early stopping ativado.

 Treinando VGG16...

Época 1/10


Train Loss: 0.0153 | Train Acc: 0.9922
Val Loss: 0.0000 | Val Acc: 1.0000
✅ Melhor modelo salvo em best_vgg16.pth
⏱️ Tempo da época: 6827.0s

Época 2/10


In [ ]:
# 7. Exportar modelos para o Google Drive
print("\n Exportando modelos para o Google Drive...")

EXPORT_PATH = "/content/drive/MyDrive/ClothingDataset/TrainedModels"
os.makedirs(EXPORT_PATH, exist_ok=True)

model_files = ["best_resnet50.pth"]

for file_name in model_files:
    source_path = os.path.join("/content/", file_name)
    destination_path = os.path.join(EXPORT_PATH, file_name)
    try:
        # Usar a função copy2 para preservar metadados
        import shutil
        shutil.copy2(source_path, destination_path)
        print(f"'{file_name}' exportado com sucesso para '{EXPORT_PATH}'")
    except FileNotFoundError:
        print(f"Erro: O arquivo '{file_name}' não foi encontrado.")
    except Exception as e:
        print(f"Erro ao exportar '{file_name}': {e}")

print("\n Exportação de modelos concluída.")

In [ ]:
# Treinar VGG16
optimizer_vgg = optim.Adam(vgg16.parameters(), lr=1e-4)
print("\n Treinando VGG16...")
vgg16, history_vgg, best_acc_vgg = train_model(
    model=vgg16,
    train_dl=train_dl,
    val_dl=val_dl,
    criterion=criterion,
    optimizer=optimizer_vgg,
    num_epochs=10,
    device=device,
    save_path="best_vgg16.pth",
    early_stopping_patience=3
)


In [ ]:
# 7. Exportar modelos para o Google Drive
print("\n Exportando modelos para o Google Drive...")

EXPORT_PATH = "/content/drive/MyDrive/ClothingDataset/TrainedModels"
os.makedirs(EXPORT_PATH, exist_ok=True)

model_files = ["best_vgg16.pth"]

for file_name in model_files:
    source_path = os.path.join("/content/", file_name)
    destination_path = os.path.join(EXPORT_PATH, file_name)
    try:
        # Usar a função copy2 para preservar metadados
        import shutil
        shutil.copy2(source_path, destination_path)
        print(f"'{file_name}' exportado com sucesso para '{EXPORT_PATH}'")
    except FileNotFoundError:
        print(f"Erro: O arquivo '{file_name}' não foi encontrado.")
    except Exception as e:
        print(f"Erro ao exportar '{file_name}': {e}")

print("\n Exportação de modelos concluída.")

In [ ]:
# Treinar EfficientNet-B0
optimizer_efficient = optim.Adam(efficientnet.parameters(), lr=1e-4)
print("\n Treinando EfficientNet-B0...")
efficientnet, history_efficient, best_acc_efficient = train_model(
    model=efficientnet,
    train_dl=train_dl,
    val_dl=val_dl,
    criterion=criterion,
    optimizer=optimizer_efficient,
    num_epochs=10,
    device=device,
    save_path="best_efficientnet.pth",
    early_stopping_patience=3
)


In [1]:
# 7. Exportar modelos para o Google Drive
print("\n Exportando modelos para o Google Drive...")

EXPORT_PATH = "/content/drive/MyDrive/ClothingDataset/TrainedModels"
os.makedirs(EXPORT_PATH, exist_ok=True)

model_files = ["best_efficientnet.pth"]

for file_name in model_files:
    source_path = os.path.join("/content/", file_name)
    destination_path = os.path.join(EXPORT_PATH, file_name)
    try:
        # Usar a função copy2 para preservar metadados
        import shutil
        shutil.copy2(source_path, destination_path)
        print(f"'{file_name}' exportado com sucesso para '{EXPORT_PATH}'")
    except FileNotFoundError:
        print(f"Erro: O arquivo '{file_name}' não foi encontrado.")
    except Exception as e:
        print(f"Erro ao exportar '{file_name}': {e}")

print("\n Exportação de modelos concluída.")


 Exportando modelos para o Google Drive...


NameError: name 'os' is not defined

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns

def evaluate_model(model, test_dl, device, class_names):
    model.eval()
    preds, true = [], []

    with torch.no_grad():
        for imgs, labels in test_dl:
            imgs, labels = imgs.to(device), labels.to(device)
            outputs = model(imgs)
            _, predicted = torch.max(outputs, 1)
            preds.extend(predicted.cpu().numpy())
            true.extend(labels.cpu().numpy())

    # Relatório de métricas
    print(classification_report(true, preds, target_names=class_names))

    # Matriz de confusão
    cm = confusion_matrix(true, preds)
    plt.figure(figsize=(8,6))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
                xticklabels=class_names, yticklabels=class_names)
    plt.xlabel("Predição")
    plt.ylabel("Verdadeiro")
    plt.title("Matriz de Confusão")
    plt.show()

# Carregar os melhores pesos salvos
resnet50.load_state_dict(torch.load("best_resnet50.pth"))
efficientnet.load_state_dict(torch.load("best_efficientnet.pth"))
vgg16.load_state_dict(torch.load("best_vgg16.pth"))

print("\n Avaliando ResNet50 no conjunto de teste...")
evaluate_model(resnet50, test_dl, device, class_names)

print("\nAvaliando EfficientNet-B0 no conjunto de teste...")
evaluate_model(efficientnet, test_dl, device, class_names)

print("\n Avaliando VGG16 no conjunto de teste...")
evaluate_model(vgg16, test_dl, device, class_names)


In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import pandas as pd

def get_metrics(model, test_dl, device, class_names):
    model.eval()
    preds, true = [], []

    with torch.no_grad():
        for imgs, labels in test_dl:
            imgs, labels = imgs.to(device), labels.to(device)
            outputs = model(imgs)
            _, predicted = torch.max(outputs, 1)
            preds.extend(predicted.cpu().numpy())
            true.extend(labels.cpu().numpy())

    acc = accuracy_score(true, preds)
    precision, recall, f1, _ = precision_recall_fscore_support(
        true, preds, average="weighted", zero_division=0
    )
    return acc, precision, recall, f1

# Avaliar cada modelo
metrics = {}
metrics["ResNet50"] = get_metrics(resnet50, test_dl, device, class_names)
metrics["EfficientNet-B0"] = get_metrics(efficientnet, test_dl, device, class_names)
metrics["VGG16"] = get_metrics(vgg16, test_dl, device, class_names)

# Criar DataFrame para organizar resultados
df_metrics = pd.DataFrame(metrics, index=["Accuracy", "Precision", "Recall", "F1-Score"]).T
print(df_metrics)

# Plot comparativo
df_metrics.plot(kind="bar", figsize=(10,6))
plt.title("Comparação de Desempenho dos Modelos no Conjunto de Teste")
plt.ylabel("Score")
plt.xticks(rotation=0)
plt.legend(loc="lower right")
plt.show()


In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import pandas as pd

def get_metrics(model, test_dl, device, class_names):
    model.eval()
    preds, true = [], []

    with torch.no_grad():
        for imgs, labels in test_dl:
            imgs, labels = imgs.to(device), labels.to(device)
            outputs = model(imgs)
            _, predicted = torch.max(outputs, 1)
            preds.extend(predicted.cpu().numpy())
            true.extend(labels.cpu().numpy())

    acc = accuracy_score(true, preds)
    precision, recall, f1, _ = precision_recall_fscore_support(
        true, preds, average="weighted", zero_division=0
    )
    return acc, precision, recall, f1

# Avaliar cada modelo
metrics = {}
metrics["ResNet50"] = get_metrics(resnet50, test_dl, device, class_names)
metrics["EfficientNet-B0"] = get_metrics(efficientnet, test_dl, device, class_names)
metrics["VGG16"] = get_metrics(vgg16, test_dl, device, class_names)

# Criar DataFrame
df_metrics = pd.DataFrame(metrics, index=["Accuracy", "Precision", "Recall", "F1-Score"]).T
print(" Métricas comparativas:\n")
print(df_metrics, "\n")

# Plot comparativo
df_metrics.plot(kind="bar", figsize=(10,6))
plt.title("Comparação de Desempenho dos Modelos no Conjunto de Teste")
plt.ylabel("Score")
plt.xticks(rotation=0)
plt.legend(loc="lower right")
plt.show()

# --- Análise automática ---
print(" Análise Automática dos Resultados:")
for metric in df_metrics.columns:
    best_model = df_metrics[metric].idxmax()
    best_score = df_metrics[metric].max()
    print(f"- {best_model} teve o melhor {metric} ({best_score:.4f})")

print("\n Interpretação geral:")
print("O modelo com maior equilíbrio entre as métricas pode ser considerado o mais robusto.")
print("Se precisão for mais importante → olhar Precision;")
print("Se cobertura for prioridade → olhar Recall;")
print("Se queremos equilíbrio → olhar F1-Score.")


In [ ]:
from sklearn.tree import DecisionTreeClassifier, plot_tree
import torch.nn as nn

# 1. Transformar modelo em extrator de features
feature_extractor = models.resnet18(weights="IMAGENET1K_V1")  # pode trocar por resnet50
feature_extractor.fc = nn.Identity()
feature_extractor = feature_extractor.to(device)
feature_extractor.eval()

# 2. Extrair embeddings
X, y = [], []

with torch.no_grad():
    for imgs, labels in tqdm(test_dl, desc="Extraindo embeddings"):
        imgs = imgs.to(device)
        feats = feature_extractor(imgs)  # vetor de features
        X.append(feats.cpu().numpy())
        y.extend(labels.numpy())

import numpy as np
X = np.vstack(X)
y = np.array(y)

print(f"Embeddings extraídos: {X.shape}, Labels: {y.shape}")

# 3. Treinar árvore de decisão
tree_clf = DecisionTreeClassifier(max_depth=3, random_state=42)
tree_clf.fit(X, y)

# 4. Plotar árvore
plt.figure(figsize=(20,10))
plot_tree(tree_clf, filled=True, feature_names=[f"f{i}" for i in range(X.shape[1])],
          class_names=class_names, rounded=True, fontsize=8)
plt.show()

# 5. Avaliar árvore
y_pred = tree_clf.predict(X)
acc = accuracy_score(y, y_pred)
print(f" Acurácia da Árvore de Decisão (mesmos dados usados no fit): {acc:.4f}")
